In [1]:
#importing necessary modules/packages
import pandas as pd
try:
    import osmsnx
except:
    %pip install osmnx
    import osmnx
import os
import geopandas as gpd
try:
    import plotly.express as px
except:
    %pip install plotly.express
    import plotly.express as px
from shapely.geometry import Point

Note: you may need to restart the kernel to use updated packages.


In [3]:
#uploading and viewing subway delay data
subway_delays = gpd.read_file('delays/subway_delay_2025.csv')
subway_delays

,_id,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
0,1,2025-01-01,2:10,Wednesday,BATHURST STATION,MUSAN,5,9,E,BD,5227
1,2,2025-01-01,2:30,Wednesday,DUNDAS STATION,MUIRS,0,0,None,YU,0
2,3,2025-01-01,2:32,Wednesday,BROADVIEW STATION,PUMST,0,0,E,BD,0
3,4,2025-01-01,2:58,Wednesday,KEELE STATION,EUSC,0,0,W,BD,5293
4,5,2025-01-01,2:58,Wednesday,COXWELL STATION,SUAE,0,0,None,BD,0
...,...,...,...,...,...,...,...,...,...,...,...
19231,19232,2025-09-30,0:24,Tuesday,YONGE BD STATION,SUO,0,0,E,BD,5217
19232,19233,2025-09-30,0:27,Tuesday,BATHURST STATION,MUIS,0,0,None,BD,0
19233,19234,2025-09-30,0:41,Tuesday,BROADVIEW STATION,SUO,0,0,None,BD,0
19234,19235,2025-09-30,0:44,Tuesday,DOWNSVIEW PARK STATION,MUIS,0,0,None,YU,0


In [4]:
#uploading and viewing subway location data
subway_locations = gpd.read_file('subway/TTC_SUBWAY_LINES_WGS84.shp')
subway_locations

,OBJECTID,ROUTE_NAME,RID,geometry
0,53420.0,LINE 1 (YONGE-UNIVERSITY),1,"LINESTRING (-79.52813 43.79677, -79.52689 43.7..."
1,53421.0,LINE 2 (BLOOR - DANFORTH),2,"LINESTRING (-79.5354 43.63781, -79.53386 43.63..."
2,53422.0,LINE 3 (SCARBOROUGH),3,"LINESTRING (-79.26332 43.73266, -79.26332 43.7..."
3,53423.0,LINE 4 (SHEPPARD),4,"LINESTRING (-79.41113 43.76145, -79.40981 43.7..."


In [5]:
#Accumulating subway delays by station + where it is bound
subway_delays_clean = subway_delays.copy()
subway_delays_clean['Min Delay'] = pd.to_numeric(subway_delays_clean['Min Delay'], errors='coerce')
subway_delays_clean['Bound'] = subway_delays_clean['Bound'].fillna('Unknown')

#summarizing delay data by stop, summing instances + minutes
subway_delay_summary = (subway_delays_clean
                 .groupby(['Line', 'Station', 'Bound'])
                 .agg(delay_count=('Min Delay', 'count'), 
                      total_delay_minutes=('Min Delay', 'sum'))
                 .reset_index())

# After creating delay_summary, filtering out rows with 0 total_delay_minutes, for reasons similar to bus delay data
subway_delay_summary = subway_delay_summary[subway_delay_summary['total_delay_minutes'] > 0]
subway_delay_summary

,Line,Station,Bound,delay_count,total_delay_minutes
1,BD,APPROACHING OLD MILL S,E,1,10
3,BD,BATHURST STATION,E,65,209
5,BD,BATHURST STATION,W,53,161
8,BD,BAY STATION,E,53,235
10,BD,BAY STATION,W,44,162
...,...,...,...,...,...
605,YU,YORK UNIVERSITY STATIO,N,23,154
607,YU,YORK UNIVERSITY STATIO,S,16,68
609,YU,YORKDALE STATION,N,38,147
611,YU,YORKDALE STATION,S,56,754


In [6]:
#I have observed that there are ttc stations that connect to several lines => for the exploratory purposes, I will
#combine data on a station level, without the Line differentiation

subway_delays_not_bound = subway_delays.copy()
subway_delays_not_bound['Min Delay'] = pd.to_numeric(subway_delays_not_bound['Min Delay'], errors='coerce')

#summarizing delay data by stop (ommitting "Bound"), summing instances + minutes
subway_delay_summary_not_bound = (subway_delays_not_bound
                 .groupby(['Line', 'Station'])
                 .agg(delay_count=('Min Delay', 'count'), 
                      total_delay_minutes=('Min Delay', 'sum'))
                 .reset_index())

# After creating delay_summary, filtering out rows with 0 total_delay_minutes, for reasons similar to bus delay data
subway_delay_summary_not_bound = subway_delay_summary_not_bound[subway_delay_summary_not_bound['total_delay_minutes'] > 0]
subway_delay_summary_not_bound.sort_values('delay_count', ascending=False) #looking at the stations with most delays

,Line,Station,delay_count,total_delay_minutes
55,BD,KENNEDY BD STATION,734,1640
223,YU,FINCH STATION,681,1496
188,YU,BLOOR STATION,667,1342
59,BD,KIPLING STATION,649,1714
217,YU,EGLINTON STATION,495,2341
...,...,...,...,...
245,YU,KING TO UNION,1,138
247,YU,KIPLING STATION,1,4
248,YU,LANSDOWNE STATION,1,12
277,YU,N/O LAWRENCE WEST STAT,1,20


In [7]:
#exporting the data for exploratory purposes
subway_delay_summary_not_bound.to_csv('TTC_subway_delays_aggregated.csv', index=False)

In [8]:
#process, similar to bus_delay and bus_location data cleaning, but now it is solely streamlining the formatting
#of stations in ttc station delay dataset
#due to the smaller number of stops, data cleaning is more manual but effective
import re #regular expression operations
def clean_station_name(name): #function for standardized ttc station names
    name = str(name).strip().upper()
    
    # removing everything after " TO " 
    if ' TO ' in name:
        name = name.split(' TO ')[0].strip()
    
    # removing everything after "STATION" 
    if 'STATION' in name:
        name = name.split('STATION')[0].strip()
    
    # handling specific aliases -- iterative manual process, where I relied on the csv file exported in the row above
    alias_mapping = {
        'VMC': 'VAUGHAN METROPOLITAN CENTRE',
        'VAUGHAN MC': 'VAUGHAN METROPOLITAN CENTRE',
        'APPROACHING OLD MILL S':'OLD MILL',
        'DAVISVILLE BUILD-UP':'DAVISVILLE',
        'DAVISVILLE YARD':'DAVISVILLE',
        'DAVISVILLE - ST. CLAIR':'DAVISVILLE',
        'GREENWOOD WYE':'GREENWOOD',
        'GREENWOOD YARD':'GREENWOOD',
        'KEELE YARD':'KEELE',
        'N/O LAWRENCE WEST': 'LAWRENCE WEST',
        'NORTH YORK CTR':'NORTH YORK CENTRE',
        'NORTH YORK CENTRE STAT':'NORTH YORK CENTRE',
        'OSGOODE POCKET':'OSGOODE',
        'OSSINGTON CENTRE TRACK':'OSSINGTON',
        'SHEPPARD YONGE':'SHEPPARD-YONGE',
        'ST. PATRICK':'ST PATRICK',
        'ST. CLAIR WEST':'ST CLAIR WEST',
        'WILSON YARD':'WILSON',
        'KENNEDY BD':'KENNEDY',
        'W/O WARDEN':'WARDEN',
        'N/O LAWRENCE WEST STAT':'LAWRENCE WEST',
       'YONGE BD':'BLOOR-YONGE',
        'BLOOR':'BLOOR-YONGE',
        'ST GEORGE BD':'ST GEORGE',
        'ST GEORGE YUS':'ST GEORGE',
        'VICTORIA PARK - WARDEN':'VICTORIA PARK',
        'FINCH TAIL':'FINCH',
        'PIONEER VILLAGE STATIO':'PIONEER VILLAGE',
        'YORK UNIVERSITY STATIO':'YORK UNIVERSITY'
    }
    
    return alias_mapping.get(name, name)


#changing the formatting of the column    
subway_delay_summary_not_bound['station_clean'] = subway_delay_summary_not_bound['Station'].apply(clean_station_name)

# aggregating the newly formatted columns
aggregated = subway_delay_summary_not_bound.groupby(['station_clean']).agg({
    'delay_count': 'sum',
    'total_delay_minutes': 'sum'
}).reset_index()

#seeing updated results with the clean data
aggregated.sort_values('total_delay_minutes', ascending = False)

,station_clean,delay_count,total_delay_minutes
4,BLOOR-YONGE,1039,2425
19,EGLINTON,495,2341
52,SHEPPARD WEST,258,1823
32,KIPLING,650,1718
69,WILSON,488,1690
...,...,...,...
50,RUNNYMEDE,128,170
26,HIGHWAY 407,88,154
3,BESSARION,74,141
55,SOUTH HOSTLER,4,18


In [9]:
#deleting the data that does not correspond to any existing stations and therefore did not aggregate
stations_to_delete = ['SOUTH HOSTLER', 'LINE 1 - YONGE UNIVERS']
aggregated = aggregated[~aggregated['station_clean'].isin(stations_to_delete)]

# resetting index after deletions
aggregated = aggregated.reset_index(drop=True)

In [10]:
aggregated.sort_values('delay_count', ascending = False)
#looking at the polished version of the dataset

,station_clean,delay_count,total_delay_minutes
4,BLOOR-YONGE,1039,2425
30,KENNEDY,767,1644
21,FINCH,685,1533
32,KIPLING,650,1718
59,ST GEORGE,502,1633
...,...,...,...
44,PIONEER VILLAGE,100,211
23,GLENCAIRN,98,282
26,HIGHWAY 407,88,154
70,YORK UNIVERSITY,86,226


In [11]:
#exporting the data
aggregated.to_csv('TTC_delays_clean_lines_aggregated.csv', index=False)

In [12]:
#uploading the stop location file and converting string longitute and latitude into point data
stops_locations = gpd.read_file('./stops.csv')
stops_locations['geometry'] = stops_locations.apply(
    lambda row: Point(row['stop_lon'], row['stop_lat']), 
    axis=1
)
#creating the gdf and reprojecting
stops_locations = gpd.GeoDataFrame(stops_locations, geometry='geometry', crs="EPSG:4326")
stops_locations

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone,wheelchair_boarding,geometry
0,662,662,Danforth Rd at Kennedy Rd,,43.714379,-79.260939,,,,,,1,POINT (-79.26094 43.71438)
1,929,929,Davenport Rd at Bedford Rd,,43.674448,-79.399659,,,,,,1,POINT (-79.39966 43.67445)
2,940,940,Davenport Rd at Dupont St,,43.675511,-79.401938,,,,,,2,POINT (-79.40194 43.67551)
3,1871,1871,Davisville Ave at Cleveland St,,43.702088,-79.378112,,,,,,1,POINT (-79.37811 43.70209)
4,11700,11700,Disco Rd at Attwell Dr,,43.701362,-79.594843,,,,,,1,POINT (-79.59484 43.70136)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9301,16806,16806,Kingston Rd at Rylander Blvd,,43.797281,-79.148975,,,,,,1,POINT (-79.14898 43.79728)
9302,16807,16807,Port Union Rd at Tilley Dr,,43.788615,-79.140347,,,,,,1,POINT (-79.14035 43.78862)
9303,16808,16808,Port Union Rd at Conference Blvd,,43.786023,-79.139246,,,,,,1,POINT (-79.13925 43.78602)
9304,16809,16809,Lawrence Ave East at Port Union Rd,,43.779626,-79.136112,,,,,,1,POINT (-79.13611 43.77963)


In [56]:
#creating a subset with only ttc stations locations, through eliminating characters that would not correspond
#to a ttc subway name
#with ttc subway name being the most consistently formatted element, the following elements will eliminate other stops
#this has also been done iteratively
stations_locations = stops_locations[
    stops_locations['stop_name'].str.contains('STATION', case=False, na=False) & #taking all the stops that have the word station in them
    ~stops_locations['stop_name'].str.contains(' AT ', case=False, na=False) & #excluding the stops with AT
    ~stops_locations['stop_name'].str.contains(' GO ', case=False, na=False) & #deleting regional stations
    ~stops_locations['stop_name'].str.contains(' RD ', case=False, na=False) & #excluding the stops with RD street type
    ~stops_locations['stop_name'].str.contains('/', case=False, na=False) #excluding the stops with special symbols
    ].copy()

#deleting duplicates
stations_locations_clean = stations_locations.drop_duplicates(
    subset=['stop_name'])

#compiling a brief dataset with most important points 
stations_locations_clean = stations_locations_clean[['stop_name','geometry']]

#deleting the word STATION to fully match with the delay data format
stations_locations_clean['stop_name'] = stations_locations_clean['stop_name'].str.upper().str.replace('STATION', '', regex=False).str.strip()
#checking the data
stations_locations_clean
#there is still a few extra stops, but this should suffise before the data merger

,stop_name,geometry
455,LAWRENCE WEST,POINT (-79.44437 43.71622)
1534,GLENCAIRN,POINT (-79.44115 43.70947)
4111,CHRISTIE,POINT (-79.41857 43.66412)
6897,UNION,POINT (-79.37921 43.64565)
7930,BATHURST,POINT (-79.41106 43.66606)
...,...,...
9049,KEELESDALE,POINT (-79.4748 43.69066)
9051,MOUNT DENNIS,POINT (-79.4877 43.68845)
9059,DON VALLEY,POINT (-79.33738 43.72166)
9080,WARDEN,POINT (-79.28001 43.70982)


In [59]:
#now that the stop location dataset is significantly cleaner, 
#we are merging ttc subway delay data with ttc location data
ttc_delay_w_coords = aggregated.merge(
    stations_locations_clean[['stop_name', 'geometry']],  
    left_on='station_clean', 
    right_on='stop_name',
    how='left'  
)

#checking the data
ttc_delay_w_coords

,station_clean,delay_count,total_delay_minutes,stop_name,geometry
0,BATHURST,236,370,BATHURST,POINT (-79.41106 43.66606)
1,BAY,165,397,NaN,None
2,BAYVIEW,126,182,NaN,None
3,BESSARION,74,141,NaN,None
4,BLOOR-YONGE,1039,2425,NaN,None
...,...,...,...,...,...
67,WILSON,488,1690,WILSON,POINT (-79.45092 43.73448)
68,WOODBINE,229,367,WOODBINE,POINT (-79.31247 43.68644)
69,YORK MILLS,215,1051,YORK MILLS,POINT (-79.40593 43.74517)
70,YORK UNIVERSITY,86,226,NaN,None


In [60]:
#there is still None values in the dataset -- let's check how many there are
none_geometry_count = len(ttc_delay_w_coords[ttc_delay_w_coords.geometry.isna()])
print(none_geometry_count)

31


#### after this point the work on the improvement of the data was not continued due to the simultaneous issues with the bus delay data, which led to us abandoning this idea, however a very plausible solution could be direct geocoding of the data.